In [1]:
import pickle

NNETWORKS = 100
NLEVELS = 3

experiment_path = 'results/SBM/SBM_MC_tanh_nnodes50_p10.5_delta_p0.5_min_weight0_bin_directedTrue_wei_directedFalse/'
network_data_path = experiment_path + 'network_data/'

results_df = pickle.load(open(experiment_path + 'average_results_df.pickle', 'rb'))
#results at criticality
results_df = results_df[results_df.alpha == 1].copy(deep=True)

timescales_df = pickle.load(open(experiment_path + 'timescales_df_inputFalse.pickle', 'rb'))

cc_df = pickle.load(open(network_data_path + 'cc_df.pickle', 'rb'))
cc_df['seed'] = [seed for level in cc_df['level'].unique() for seed in range(len(cc_df[cc_df['level'] == level]))]

freq_func_motif_df = pickle.load(open(network_data_path + 'freq_func_motif_df.pickle', 'rb'))
reordered_motif_dict = {}
for motif in range(1, 14):
    reordered_motif_dict[motif] = []
    for network in range(NNETWORKS):
        for level in range(NLEVELS):
            reordered_motif_dict[motif].append(freq_func_motif_df[freq_func_motif_df['motif'] == motif].F.values[network + level * NNETWORKS])
            
ncycles_df = pickle.load(open(network_data_path + 'ncycles_df.pickle', 'rb'))
reordered_ncycles_dict = {}
for cycle_len in range(3, 6):
    reordered_ncycles_dict[cycle_len] = []
    for network in range(NNETWORKS):
        for level in range(NLEVELS):
            reordered_ncycles_dict[cycle_len].append(ncycles_df[ncycles_df['cycle_len'] == cycle_len].n_cycles.values[network + level * NNETWORKS])

In [2]:
#Ensure all dataframes are sorted the same way
results_df = results_df.sort_values(['seed', 'level'])
timescales_df = timescales_df.sort_values(['seed', 'level'])
cc_df = cc_df.sort_values(['seed', 'level'])

#Merge the dataframes
results_df['timescale'] = timescales_df['timescale'].values
results_df['cc'] = cc_df['cc'].values
for motif in range(1, 14):
    results_df['motif_' + str(motif)] = reordered_motif_dict[motif]
for cycle_len in range(3, 6):
    results_df['ncycles_' + str(cycle_len)] = reordered_ncycles_dict[cycle_len]

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

from score_plotting_utils import create_fig_path

sns.set_style("ticks")
sns.set(context=None, style=None, palette=None, font_scale=5, color_codes=None)
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams.update({'font.size': 15})
plt.rcParams['legend.fontsize'] = 15

PuBu = cm.get_cmap('PuBu')
GnBu = cm.get_cmap('GnBu')
PuRd = cm.get_cmap('PuRd')
palette = [PuBu(0.6), GnBu(0.6), PuRd(0.6)]

In [4]:
#TIMESCALE VARIABILITY

from scipy.stats import spearmanr

import os
fig_path = create_fig_path(experiment_path)

ax = sns.regplot(data=results_df, x='timescale', y='score', color='grey', scatter=False)
sns.scatterplot(data=results_df, x='timescale', y='score', hue='level', palette=palette, legend=False, ax=ax)

spearman = spearmanr(results_df['timescale'], results_df['score'])
print(f'Spearman\'s rho: {spearman.correlation:.2f}, p-value: {spearman.pvalue:.2e}')

ax.text(0.35, 0.1, "Spearman\'s rho: {:.2f}".format(spearman.correlation), ha='center', va='center', transform=ax.transAxes)
ax.set_box_aspect(1)

ax.set_xlabel('Timescale variability')
ax.set_ylabel('Memory capacity')

fig = ax.get_figure()
fig.savefig(os.path.join(fig_path, 'timescale_var_x_MC.png'), dpi=300, bbox_inches='tight')
fig.savefig(os.path.join(fig_path, 'timescale_var_x_MC.svg'), dpi=300, bbox_inches='tight')
plt.close(fig)

Spearman's rho: 0.52, p-value: 3.81e-22


In [5]:
#CLUSTERING COEFFICIENT

ax = sns.regplot(data=results_df, x='cc', y='score', color='grey', scatter=False)
sns.scatterplot(data=results_df, x='cc', y='score', hue='level', palette=palette, legend=False, ax=ax)

spearman = spearmanr(results_df['cc'], results_df['score'])
print(f'Spearman\'s rho: {spearman.correlation:.2f}, p-value: {spearman.pvalue:.2e}')

ax.text(0.35, 0.1, "Spearman\'s rho: {:.2f}".format(spearman.correlation), ha='center', va='center', transform=ax.transAxes)
ax.set_box_aspect(1)

ax.set_xlabel('Clustering coefficient')
ax.set_ylabel('Memory capacity')

fig = ax.get_figure()
fig.savefig(os.path.join(fig_path, 'cc_x_MC.png'), dpi=300, bbox_inches='tight')
fig.savefig(os.path.join(fig_path, 'cc_x_MC.svg'), dpi=300, bbox_inches='tight')
plt.close(fig)

Spearman's rho: 0.68, p-value: 1.27e-41


In [6]:
#MOTIFS

#motif 1-3 grid
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for i in range(3):

    print('Motif ' + str(i + 1))

    sns.regplot(data=results_df, x='motif_' + str(i + 1), y='score', color='grey', scatter=False, ax=axs[i])
    sns.scatterplot(data=results_df, x='motif_' + str(i + 1), y='score', hue='level', palette=palette, legend=False, ax=axs[i])

    spearman = spearmanr(results_df['motif_' + str(i + 1)], results_df['score'])
    print(f'Spearman\'s rho: {spearman.correlation:.2f}, p-value: {spearman.pvalue:.2e}')

    axs[i].text(0.35, 0.1, "Spearman\'s rho: {:.2f}".format(spearman.correlation), ha='center', va='center', transform=axs[i].transAxes)
    axs[i].set_box_aspect(1)
    axs[i].set_xlabel('Motif ' + str(i + 1))
    axs[i].set_ylabel('Memory capacity')

fig.savefig(os.path.join(fig_path, 'motifs123_x_MC.png'), dpi=300, bbox_inches='tight')
fig.savefig(os.path.join(fig_path, 'motifs123_x_MC.svg'), dpi=300, bbox_inches='tight')
plt.close(fig)

#remaining motifs
fig, axs = plt.subplots(2, 5, figsize=(25, 10))
for i in range(10):

    print('Motif ' + str(i + 4))

    sns.regplot(data=results_df, x='motif_' + str(i + 4), y='score', color='grey', scatter=False, ax=axs[i // 5, i % 5])
    sns.scatterplot(data=results_df, x='motif_' + str(i + 4), y='score', hue='level', palette=palette, legend=False, ax=axs[i // 5, i % 5])

    spearman = spearmanr(results_df['motif_' + str(i + 4)], results_df['score'])
    print(f'Spearman\'s rho: {spearman.correlation:.2f}, p-value: {spearman.pvalue:.2e}')

    axs[i // 5, i % 5].text(0.35, 0.1, "Spearman\'s rho: {:.2f}".format(spearman.correlation), ha='center', va='center', transform=axs[i // 5, i % 5].transAxes)
    axs[i // 5, i % 5].set_box_aspect(1)
    axs[i // 5, i % 5].set_xlabel('Motif ' + str(i + 4))
    axs[i // 5, i % 5].set_ylabel('Memory capacity')

fig.savefig(os.path.join(fig_path, 'motifs_x_MC.png'), dpi=300, bbox_inches='tight')
fig.savefig(os.path.join(fig_path, 'motifs_x_MC.svg'), dpi=300, bbox_inches='tight')
plt.close(fig)

Motif 1
Spearman's rho: -0.67, p-value: 6.43e-40
Motif 2
Spearman's rho: -0.66, p-value: 2.31e-38
Motif 3
Spearman's rho: -0.61, p-value: 1.85e-32
Motif 4
Spearman's rho: 0.61, p-value: 1.83e-31
Motif 5
Spearman's rho: 0.68, p-value: 2.30e-42
Motif 6
Spearman's rho: 0.63, p-value: 5.56e-34
Motif 7
Spearman's rho: 0.68, p-value: 2.92e-42
Motif 8
Spearman's rho: 0.67, p-value: 1.59e-40
Motif 9
Spearman's rho: 0.62, p-value: 8.43e-34
Motif 10
Spearman's rho: 0.67, p-value: 5.89e-40
Motif 11
Spearman's rho: 0.65, p-value: 3.96e-38
Motif 12
Spearman's rho: 0.65, p-value: 1.43e-37
Motif 13
Spearman's rho: 0.48, p-value: 7.83e-19


In [7]:
#CYCLES

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for i in range(3):

    print('Cycle length: ' + str(i + 1))

    sns.regplot(data=results_df, x='ncycles_' + str(i + 3), y='score', color='grey', scatter=False, ax=axs[i])
    sns.scatterplot(data=results_df, x='ncycles_' + str(i + 3), y='score', hue='level', palette=palette, legend=False, ax=axs[i])

    spearman = spearmanr(results_df['ncycles_' + str(i + 3)], results_df['score'])
    print(f'Spearman\'s rho: {spearman.correlation:.2f}, p-value: {spearman.pvalue:.2e}')

    axs[i].text(0.35, 0.1, "Spearman\'s rho: {:.2f}".format(spearman.correlation), ha='center', va='center', transform=axs[i].transAxes)
    axs[i].set_box_aspect(1)
    axs[i].set_xlabel('Cycle length: ' + str(i + 1))
    axs[i].set_ylabel('Memory capacity')

fig.savefig(os.path.join(fig_path, 'ncycles_x_MC.png'), dpi=300, bbox_inches='tight')
fig.savefig(os.path.join(fig_path, 'ncycles_x_MC.svg'), dpi=300, bbox_inches='tight')
plt.close(fig)

Cycle length: 1
Spearman's rho: 0.66, p-value: 4.37e-39
Cycle length: 2
Spearman's rho: 0.56, p-value: 8.59e-26
Cycle length: 3
Spearman's rho: 0.05, p-value: 3.61e-01
